In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
dr18q = pd.read_csv('dr18_z5.csv')[['ra', 'dec', 'z', 'plate', 'mjd', 'fiberID']]
dr18q

,ra,dec,z,plate,mjd,fiberID
0,24.197015,-9.811836,6.471043,1915,53612,174
1,172.038220,32.725923,5.270376,1974,53430,341
2,21.136845,31.321050,6.588362,2061,53711,36
3,138.757160,24.303370,6.514880,2291,53714,309
4,4.311119,-10.015412,5.050612,652,52138,152
...,...,...,...,...,...,...
2480,130.498150,20.824962,6.474940,11097,58522,878
2481,130.228900,22.515871,5.571056,11106,58511,689
2482,149.328010,28.527658,5.437895,11349,58441,723
2483,177.109030,30.338698,5.134676,11391,58526,190


In [3]:
dr16q = pd.read_csv('manual_z5_full_20230130.csv')[[
    'RA', 'DEC', 'plate', 'mjd', 'fiber', 
    'Comment', 'Interesting_Score_max10', 'Owner', 'Vizzspec', 'Vizclass', 'Z'
]]
dr16q.rename(columns={'RA': 'ra', 'DEC': 'dec', 'fiber': 'fiberID', 'Z': 'DR16q_z'}, inplace=True)
dr16q.insert(2, 'z', [np.nan] * dr16q.shape[0])
dr16q['ra'] = list(map(lambda x: float(x.replace(',', '.')), dr16q['ra']))
dr16q['dec'] = list(map(lambda x: float(x.replace(',', '.')), dr16q['dec']))
dr16q['DR16q_z'] = list(map(lambda x: float(x.replace(',', '.')), dr16q['DR16q_z']))
dr16q

,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z
0,31.213077,25.667587,NaN,7692,57064,878,луч звезды,0.0,2.0,NaN,5,7.011245
1,228.237805,57.835578,NaN,8423,58198,294,NaN,3.0,NaN,NaN,5,7.011245
2,15.081000,9.836346,NaN,11055,58429,965,Interesting BAL,5.0,NaN,NaN,7,7.011245
3,165.982014,35.926578,NaN,4627,55626,543,NaN,1.0,2.0,NaN,5,7.006674
4,121.888749,31.988892,NaN,10293,58132,703,NaN,1.0,NaN,NaN,6,7.004498
...,...,...,...,...,...,...,...,...,...,...,...,...
650,258.863753,38.866850,NaN,10763,58395,182,луч звезды,0.0,2.0,NaN,5,7.011245
651,130.949372,20.631246,NaN,9565,58127,536,NaN,4.0,NaN,"0,2275433249",1,5.608535
652,149.328006,28.527658,NaN,11349,58441,723,NaN,1.0,2.0,NaN,6,5.437960
653,244.427230,32.376226,NaN,10918,58254,746,NaN,5.0,2.0,NaN,7,5.190181


In [5]:
def angular_dist(ra_01, dec_01, ra_02, dec_02):
#     ra_01, dec_01 = x[0], x[1]
#     ra_02, dec_02 = y[0], y[1]
    ra_01 = ra_01 * np.pi / 180
    ra_02 = ra_02 * np.pi / 180
    dec_01 = dec_01 * np.pi / 180
    dec_02 = dec_02 * np.pi / 180
    return np.arccos(np.minimum(
        np.sin(dec_01) * np.sin(dec_02) +
        np.cos(dec_01) * np.cos(dec_02) * np.cos(ra_01 - ra_02),
        1
    )) * 180 / (np.pi)

In [6]:
dr16q.columns

Index(['ra', 'dec', 'z', 'plate', 'mjd', 'fiberID', 'Comment',
       'Interesting_Score_max10', 'Owner', 'Vizzspec', 'Vizclass', 'DR16q_z'],
      dtype='object')

In [7]:
Commet_c, Interesting_Score_max10_c, Owner_c, Vizzspec_c, Vizclass_c, DR16q_z_c = [], [], [], [], [], []
drop_idx = []
for i in tqdm(range(dr18q.shape[0])):
    row_01 = dr18q.iloc[i]
    min_dist = 10000000
    idx = None
    Comment, Interesting_Score_max10, Owner, Vizzspec, Vizclass, DR16q_z = None, None, None, None, None, None
    for j in range(dr16q.shape[0]):
        row_02 = dr16q.iloc[j]
        dist = angular_dist(
            row_01['ra'], row_01['dec'], 
            row_02['ra'], row_02['dec']
        ) * 3600
        if (dist <= 3) and (dist <= min_dist):
            min_dist = dist
            idx = j
            Commet, Interesting_Score_max10, Owner, Vizzspec, Vizclass, DR16q_z \
                = row_02['Comment'], row_02['Interesting_Score_max10'], row_02['Owner'], \
                  row_02['Vizzspec'], row_02['Vizclass'], row_02['DR16q_z']
    Commet_c.append(Comment)
    Interesting_Score_max10_c.append(Interesting_Score_max10)
    Owner_c.append(Owner)
    Vizzspec_c.append(Vizzspec)
    Vizclass_c.append(Vizclass)
    DR16q_z_c.append(DR16q_z)
    if idx is not None:
        drop_idx.append(idx)
dr18q['Comment'], dr18q['Interesting_Score_max10'], dr18q['Owner'], \
dr18q['Vizzspec'], dr18q['Vizclass'], dr18q['DR16q_z'] \
    = Commet_c, Interesting_Score_max10_c, Owner_c, Vizzspec_c, Vizclass_c, DR16q_z_c

100%|███████████████████████████████████████| 2485/2485 [02:04<00:00, 20.04it/s]


In [8]:
dr18q

,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z
0,24.197015,-9.811836,6.471043,1915,53612,174,None,NaN,NaN,None,NaN,NaN
1,172.038220,32.725923,5.270376,1974,53430,341,None,NaN,NaN,None,NaN,NaN
2,21.136845,31.321050,6.588362,2061,53711,36,None,NaN,NaN,None,NaN,NaN
3,138.757160,24.303370,6.514880,2291,53714,309,None,NaN,NaN,None,NaN,NaN
4,4.311119,-10.015412,5.050612,652,52138,152,None,6.0,2.0,NaN,0.0,5.019688
...,...,...,...,...,...,...,...,...,...,...,...,...
2480,130.498150,20.824962,6.474940,11097,58522,878,None,NaN,NaN,None,NaN,NaN
2481,130.228900,22.515871,5.571056,11106,58511,689,None,1.0,2.0,NaN,6.0,5.570852
2482,149.328010,28.527658,5.437895,11349,58441,723,None,1.0,2.0,NaN,6.0,5.437960
2483,177.109030,30.338698,5.134676,11391,58526,190,None,6.0,1.0,"5,13",0.0,5.130000


In [9]:
(~dr18q['Vizclass'].isna()).sum()

251

In [13]:
dr16q_copy = dr16q.copy()
dr16q_copy.drop(index=drop_idx, inplace=True)
dr16q_copy

,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z
1,228.237805,57.835578,NaN,8423,58198,294,NaN,3.0,NaN,NaN,5,7.011245
3,165.982014,35.926578,NaN,4627,55626,543,NaN,1.0,2.0,NaN,5,7.006674
4,121.888749,31.988892,NaN,10293,58132,703,NaN,1.0,NaN,NaN,6,7.004498
6,157.288756,33.529002,NaN,10271,58497,528,NaN,NaN,NaN,NaN,5,6.982019
7,146.931183,35.425663,NaN,10229,58441,992,NaN,NaN,2.0,NaN,5,6.971168
...,...,...,...,...,...,...,...,...,...,...,...,...
637,346.332993,33.063309,NaN,11636,58462,153,NaN,1.0,2.0,NaN,7,6.923885
638,10.740187,-5.396737,NaN,7905,57666,762,NaN,1.0,NaN,NaN,16,5.596944
639,129.148445,31.661450,NaN,10664,58464,752,NaN,0.0,2.0,NaN,5,6.533200
643,131.203669,54.801972,NaN,7374,56751,60,NaN,NaN,NaN,NaN,5,5.599056


In [18]:
dr18q_16q = pd.concat([dr18q, dr16q_copy])
dr18q_16q.reset_index(drop=True, inplace=True)
dr18q_16q

,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z
0,24.197015,-9.811836,6.471043,1915,53612,174,None,NaN,NaN,None,NaN,NaN
1,172.038220,32.725923,5.270376,1974,53430,341,None,NaN,NaN,None,NaN,NaN
2,21.136845,31.321050,6.588362,2061,53711,36,None,NaN,NaN,None,NaN,NaN
3,138.757160,24.303370,6.514880,2291,53714,309,None,NaN,NaN,None,NaN,NaN
4,4.311119,-10.015412,5.050612,652,52138,152,None,6.0,2.0,NaN,0.0,5.019688
...,...,...,...,...,...,...,...,...,...,...,...,...
2884,346.332993,33.063309,NaN,11636,58462,153,NaN,1.0,2.0,NaN,7.0,6.923885
2885,10.740187,-5.396737,NaN,7905,57666,762,NaN,1.0,NaN,NaN,16.0,5.596944
2886,129.148445,31.661450,NaN,10664,58464,752,NaN,0.0,2.0,NaN,5.0,6.533200
2887,131.203669,54.801972,NaN,7374,56751,60,NaN,NaN,NaN,NaN,5.0,5.599056


In [20]:
from astropy.table import Table

In [21]:
t = Table.read(f'DR14Q_v4_4.fits')
dr14q = t[['RA', 'DEC', 'Z']].to_pandas()
dr14q = dr14q[dr14q['Z'] > 5]
dr14q

,RA,DEC,Z
3688,1.468054,-0.115457,5.855000
7277,2.938967,23.394451,5.722000
7538,3.030954,0.214939,5.979728
10811,4.311153,-10.015396,5.010500
14804,5.877779,-0.310170,5.040000
...,...,...,...
485386,334.183406,0.230033,5.010000
486488,335.077095,-1.029748,5.605000
489375,337.188110,-7.965384,5.141700
505736,348.068494,1.014343,5.068000


In [24]:
cor_idx = []
DR14q_z_c = []
for i in tqdm(range(dr18q_16q.shape[0])):
    row_01 = dr18q_16q.iloc[i]
    min_dist = 10000000
    idx = None
    DR14q_z = None
    for j in range(dr14q.shape[0]):
        row_02 = dr14q.iloc[j]
        dist = angular_dist(
            row_01['ra'], row_01['dec'], 
            row_02['RA'], row_02['DEC']
        ) * 3600
        if (dist <= 3) and (dist <= min_dist):
            min_dist = dist
            idx = j
            DR14q_z = row_02['Z']
    DR14q_z_c.append(DR14q_z)
    if idx is not None:
        cor_idx.append(idx)
dr18q_16q['DR14q_z'] = DR14q_z_c

100%|██████████████████████████████████████| 2889/2889 [00:13<00:00, 222.22it/s]


In [25]:
dr18q_16q

,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z,DR14q_z
0,24.197015,-9.811836,6.471043,1915,53612,174,None,NaN,NaN,None,NaN,NaN,NaN
1,172.038220,32.725923,5.270376,1974,53430,341,None,NaN,NaN,None,NaN,NaN,NaN
2,21.136845,31.321050,6.588362,2061,53711,36,None,NaN,NaN,None,NaN,NaN,NaN
3,138.757160,24.303370,6.514880,2291,53714,309,None,NaN,NaN,None,NaN,NaN,NaN
4,4.311119,-10.015412,5.050612,652,52138,152,None,6.0,2.0,NaN,0.0,5.019688,5.0105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,346.332993,33.063309,NaN,11636,58462,153,NaN,1.0,2.0,NaN,7.0,6.923885,NaN
2885,10.740187,-5.396737,NaN,7905,57666,762,NaN,1.0,NaN,NaN,16.0,5.596944,NaN
2886,129.148445,31.661450,NaN,10664,58464,752,NaN,0.0,2.0,NaN,5.0,6.533200,NaN
2887,131.203669,54.801972,NaN,7374,56751,60,NaN,NaN,NaN,NaN,5.0,5.599056,NaN


In [26]:
(~dr18q_16q['DR14q_z'].isna()).sum()

79

In [27]:
len(np.unique(cor_idx))

79

In [28]:
links = []
plates = list(dr18q_16q['plate'])
mjds = list(dr18q_16q['mjd'])
fibers = list(dr18q_16q['fiberID'])
links = []
for i in range(len(plates)):
    links.append(f'http://skyserver.sdss.org/dr17/VisualTools/quickobj?plate={plates[i]}&mjd={mjds[i]}&fiber={fibers[i]}')
dr18q_16q['link'] = links 

In [29]:
dr18q_16q

,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z,DR14q_z,link
0,24.197015,-9.811836,6.471043,1915,53612,174,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
1,172.038220,32.725923,5.270376,1974,53430,341,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2,21.136845,31.321050,6.588362,2061,53711,36,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
3,138.757160,24.303370,6.514880,2291,53714,309,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
4,4.311119,-10.015412,5.050612,652,52138,152,None,6.0,2.0,NaN,0.0,5.019688,5.0105,http://skyserver.sdss.org/dr17/VisualTools/qui...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,346.332993,33.063309,NaN,11636,58462,153,NaN,1.0,2.0,NaN,7.0,6.923885,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2885,10.740187,-5.396737,NaN,7905,57666,762,NaN,1.0,NaN,NaN,16.0,5.596944,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2886,129.148445,31.661450,NaN,10664,58464,752,NaN,0.0,2.0,NaN,5.0,6.533200,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2887,131.203669,54.801972,NaN,7374,56751,60,NaN,NaN,NaN,NaN,5.0,5.599056,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...


In [30]:
dr18q_16q.insert(0, 'idx', range(dr18q_16q.shape[0]))
dr18q_16q

,idx,ra,dec,z,plate,mjd,fiberID,Comment,Interesting_Score_max10,Owner,Vizzspec,Vizclass,DR16q_z,DR14q_z,link
0,0,24.197015,-9.811836,6.471043,1915,53612,174,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
1,1,172.038220,32.725923,5.270376,1974,53430,341,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2,2,21.136845,31.321050,6.588362,2061,53711,36,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
3,3,138.757160,24.303370,6.514880,2291,53714,309,None,NaN,NaN,None,NaN,NaN,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
4,4,4.311119,-10.015412,5.050612,652,52138,152,None,6.0,2.0,NaN,0.0,5.019688,5.0105,http://skyserver.sdss.org/dr17/VisualTools/qui...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,2884,346.332993,33.063309,NaN,11636,58462,153,NaN,1.0,2.0,NaN,7.0,6.923885,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2885,2885,10.740187,-5.396737,NaN,7905,57666,762,NaN,1.0,NaN,NaN,16.0,5.596944,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2886,2886,129.148445,31.661450,NaN,10664,58464,752,NaN,0.0,2.0,NaN,5.0,6.533200,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...
2887,2887,131.203669,54.801972,NaN,7374,56751,60,NaN,NaN,NaN,NaN,5.0,5.599056,NaN,http://skyserver.sdss.org/dr17/VisualTools/qui...


In [31]:
dr18q_16q.to_csv('dr18q_z5.csv', index=False)